In [1]:
import numpy as np
import math
import pandas as pd
import xgboost
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.svm import SVR
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, GradientBoostingRegressor
import copy

In [2]:
data_path = "/content/drive/MyDrive/Colab Notebooks/DS-nanodegree/Project 1/data/clean_data/data_selected.csv"

# data_path = "/content/drive/MyDrive/Colab Notebooks/DS-nanodegree/Project 1/data/paper-data/data_selected_2.csv"

In [3]:
def print_result(model, x_val, y_val):
    y_pred_val = model.predict(x_val)
    print('---Test---')
    print('MAE: ', metrics.mean_absolute_error(y_val, y_pred_val))
    print('MSE: ', metrics.mean_squared_error(y_val, y_pred_val))
    print('R2: ', metrics.r2_score(y_val, y_pred_val))

In [4]:
df = pd.read_csv(data_path, index_col=0)

In [5]:
df.head()

,host_since,host_response_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,longitude,accommodates,bathrooms,bedrooms,beds,price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,number_of_reviews,number_of_reviews_ltm,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,require_guest_phone_verification,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,email,...,Singlelevelhome,Smartlock,Smokedetector,Smokingallowed,Stove,Suitableforevents,TV,Toiletpaper,Washer,Well-litpathtoentrance,Wheelchairaccessible,Wideclearancetoshower,Widedoorwaytoguestbathroom,Wideentranceforguests,Wideentryway,Wifi,translationmissing:en.hosting_amenity_49,translationmissing:en.hosting_amenity_50,Apartment,Bed and breakfast,Condominium,Guesthouse,Hostel,House,Serviced apartment,Townhouse,Real Bed,Entire home/apt,Hotel room,Private room,flexible,moderate,strict_14_with_grace_period,super_strict_30,super_strict_60,0,a few days or more,within a day,within a few hours,within an hour
0,4517.0,100,1,3.0,1,1,-0.11732,4,1.0,1.0,1.0,4.477337,5.991465,3.401197,2,2.995732,2,180,2,2,180,180,2.0,180.0,184,1,92.0,9.0,9.0,10.0,10.0,9.0,9.0,1,1,2,0,0,1.59,1,...,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,1
1,4473.0,100,0,4.0,1,0,-0.11121,2,1.0,1.0,0.0,4.174387,4.605170,2.708050,1,2.708050,1,29,1,1,29,29,1.0,29.0,16,2,96.0,9.0,10.0,9.0,10.0,9.0,9.0,0,0,1,2,0,0.14,1,...,0,0,1,1,1,0,1,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1
2,4454.0,100,1,1.0,1,1,-0.16898,2,1.0,1.0,1.0,4.605170,5.010635,3.912023,2,-55.000000,30,50,30,30,50,50,30.0,50.0,87,6,96.0,10.0,10.0,10.0,10.0,10.0,9.0,1,1,1,0,0,0.73,1,...,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0
3,4424.0,98,1,18.0,1,1,-0.14002,6,2.0,3.0,4.0,5.703782,5.857933,4.174387,4,2.302585,3,365,2,3,365,365,3.0,365.0,41,5,94.0,10.0,9.0,9.0,9.0,10.0,9.0,1,0,15,0,0,0.40,1,...,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,1
4,4522.0,80,0,0.0,1,1,-0.16376,2,1.5,1.0,1.0,5.164786,5.521461,4.248495,2,-55.000000,30,1125,30,30,1125,1125,30.0,1125.0,93,1,98.0,10.0,10.0,10.0,10.0,9.0,9.0,0,1,1,0,0,0.86,1,...,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,1,0,0,0,0,0,1,0,0


## Linear Regression

In [6]:
def Linear(x_train, y_train):
    model = LinearRegression()
    model.fit(x_train, y_train)
    return model

# df = pd.read_csv(data_path)
dataset = df.loc[:, df.columns != 'price']

X = dataset.loc[:, dataset.columns != 'city'].values
Y = df.loc[:, df.columns == 'price'].values

min_max_scaler = preprocessing.MinMaxScaler()
X_scale = min_max_scaler.fit_transform(X)

X_train, X_val, Y_train, Y_val = train_test_split(X_scale, Y, test_size=0.3)

linear_model = Linear(X_train, Y_train)
print_result(linear_model, X_train, Y_train)
print_result(linear_model, X_val, Y_val)

---Test---
MAE:  0.3395667872524401
MSE:  0.86153801717656
R2:  0.32915308115693565
---Test---
MAE:  1192540.1481531283
MSE:  3.615250515487971e+16
R2:  -2.2634965360496148e+16


## Gradient Boosting

In [7]:
def GradientBoosting(x_train, y_train):
    grad = GradientBoostingRegressor(n_estimators=1000, verbose=1)
    grad.fit(x_train, y_train)
    return grad

grad = GradientBoosting(X_train, Y_train.ravel())
print_result(grad, X_train, Y_train)
print_result(grad, X_val, Y_val)

      Iter       Train Loss   Remaining Time 
         1           1.2186            7.08m
         2           1.1634            6.75m
         3           1.1178            6.67m
         4           1.0811            6.62m
         5           1.0502            7.29m
         6           1.0241            8.02m
         7           1.0028            8.53m
         8           0.9843            8.79m
         9           0.9688            9.09m
        10           0.9558            9.30m
        20           0.8872           10.84m
        30           0.8573           11.89m
        40           0.8188           10.99m
        50           0.8094            9.97m
        60           0.7978            9.27m
        70           0.7920            8.74m
        80           0.7828            8.35m
        90           0.7709            8.00m
       100           0.7612            7.73m
       200           0.6620            6.11m
       300           0.6045            5.14m
       40

## Support Vector Regression

In [8]:
def svr(x_train, y_train):
    regressor = SVR(kernel='rbf', max_iter=3000, verbose=1)
    regressor.fit(x_train, y_train)
    return regressor

In [9]:
df = pd.read_csv(data_path, index_col=0)

dataset = df.loc[:, df.columns != 'price']

X = dataset.loc[:, dataset.columns != 'city'].values
Y = df.loc[:, df.columns == 'price'].values

sc_X = preprocessing.StandardScaler()
sc_Y = preprocessing.StandardScaler()
X = sc_X.fit_transform(X)
Y = sc_Y.fit_transform(Y)

X_train, X_val_and_test, Y_train, Y_val_and_test = train_test_split(X, Y, test_size=0.3)
X_val, X_test, Y_val, Y_test = train_test_split(X_val_and_test, Y_val_and_test, test_size=0.5)

svr_model = svr(X_train, Y_train.ravel())
print_result(svr_model, X_train, Y_train)
print_result(svr_model, X_val, Y_val)
print_result(svr_model, X_test, Y_test)

[LibSVM]

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=3000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,


---Test---
MAE:  0.26211088167934143
MSE:  0.7527055141201632
R2:  0.3199545494288284
---Test---
MAE:  0.2788878776778451
MSE:  0.542459545077926
R2:  0.3647644078043898
---Test---
MAE:  0.27994399175973084
MSE:  0.34674160618284156
R2:  0.46422689372683


## XGBoost

In [10]:
def XGB(x_train, y_train):
    xgb = xgboost.XGBRegressor(learning_rate=0.01, n_estimators=500, max_depth=15,
                                subsample=0.8, colsample_bytree=0.5, gamma=1)

    xgb.fit(x_train, y_train, verbose=True)
    return xgb

In [11]:
df = pd.read_csv(data_path, index_col=0)

dataset = df.loc[:, df.columns != 'price']

X = dataset.loc[:, dataset.columns != 'city'].values
Y = df.loc[:, df.columns == 'price'].values

X_train, X_val_and_test, Y_train, Y_val_and_test = train_test_split(X, Y, test_size=0.3)
X_val, X_test, Y_val, Y_test = train_test_split(X_val_and_test, Y_val_and_test, test_size=0.5)

xgb = XGB(X_train, Y_train)
print_result(xgb, X_train, Y_train)
print_result(xgb, X_val, Y_val)
print_result(xgb, X_test, Y_test)

[23:39:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
---Test---
MAE:  0.22765166039720386
MSE:  0.10778060932878526
R2:  0.9198373549845196
---Test---
MAE:  0.2844741363520254
MSE:  0.7235466027105808
R2:  0.39310293921413264
---Test---
MAE:  0.2913305700089631
MSE:  1.2581852014979205
R2:  0.26883758010235626


## k-means with Gradient Boosting + KFold

In [12]:
def gradient(X_train, y_train, X_val, y_val):
    n_clusters = 8
    kmeans = KMeans(n_clusters=n_clusters, verbose=0).fit(X_train)

    c_train = kmeans.predict(X_train)
    c_pred = kmeans.predict(X_val)
    centroids = kmeans.cluster_centers_
    y_val_stats = None
    predicted_values = None
    y_train_stats = None
    labels_stats = None
    for i in range(n_clusters):
        print(c_train)
        print('--------analyzing cluster %d--------' %i)
        train_mask = c_train==i
        std_train = np.std(y_train[train_mask])
        mean_train = np.mean(y_train[train_mask])
        print("# examples & price mean & std for training set within cluster %d is:(%d, %.2f, %.2f)" %(i, train_mask.sum(), np.float(mean_train), np.float(std_train)))
        pred_mask = c_pred==i
        std_pred = np.std(y_val[pred_mask])
        mean_pred = np.mean(y_val[pred_mask])
        print("# examples & price mean & std for validation set within cluster %d is:(%d, %.2f, %.2f)" %(i, pred_mask.sum(), np.float(mean_pred), np.float(std_pred)))
        if pred_mask.sum() == 0:
            print('Zero membered test set! Skipping the test and training validation.')
            continue

        # GB
        regr = GradientBoostingRegressor(n_estimators=10, verbose=0)
        regr.fit(X_train[train_mask], y_train[train_mask])

        cv = KFold(n_splits=10, random_state=42, shuffle=True)
        scores = cross_val_score(regr, X_train[train_mask], y_train[train_mask], cv=cv)
        print('Accuracy cluster %d: %0.9f (%0.9f)' % (i, np.mean(scores), np.std(scores)))

        labels_pred = regr.predict(X_train[train_mask].values)
        y_pred = regr.predict(X_val[pred_mask].values)
        if (y_val_stats is None):
            y_val_stats = copy.deepcopy(y_val[pred_mask])
            y_train_stats = copy.deepcopy(y_train[train_mask])
            predicted_values = copy.deepcopy(y_pred)
            labels_stats = copy.deepcopy(labels_pred)

        else:
            y_val_stats = y_val_stats.append(y_val[pred_mask])
            y_train_stats = y_train_stats.append(y_train[train_mask])
            predicted_values = np.append(predicted_values, y_pred)
            labels_stats = np.append(labels_stats, labels_pred)
        print('--------Finished analyzing cluster %d--------' %i)

    print('--------------------TEST--------------------')
    print("Mean absolute error: ",
          metrics.mean_absolute_error(y_val_stats, predicted_values))
    print("Median absolute error: ",
          metrics.median_absolute_error(y_val_stats, predicted_values))
    print("Mean squared error: ", metrics.mean_squared_error(
        y_val_stats, predicted_values))
    print("R2: ", metrics.r2_score(y_val_stats, predicted_values))
    print('--------------------TRAIN--------------------')
    print("Mean absolute error: ",
        metrics.mean_absolute_error(y_train_stats, labels_stats))
    print("Median absolute error: ",
        metrics.median_absolute_error(y_train_stats, labels_stats))
    print("Mean squared error: ", metrics.mean_squared_error(
        y_train_stats, labels_stats))
    print("R2: ", metrics.r2_score(y_train_stats, labels_stats))


    return c_pred, centroids

In [13]:
df = pd.read_csv(data_path, index_col=0)

In [14]:
dataset = df.loc[:, df.columns != 'price']
dataset = df.loc[:, df.columns != 'city']

In [15]:
X_normalized = dataset.values.astype(float)
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(X_normalized)
x = pd.DataFrame(x_scaled, columns=dataset.columns)

In [16]:
X_train, X_val_and_test, Y_train, Y_val_and_test = train_test_split(x, df['price'], test_size=0.3)
X_val, X_test, Y_val, Y_test = train_test_split(X_val_and_test, Y_val_and_test, test_size=0.5)

In [17]:
c_pred, centroids = gradient(X_train, Y_train, X_test, Y_test)

[1 5 1 ... 2 5 7]
--------analyzing cluster 0--------
# examples & price mean & std for training set within cluster 0 is:(3256, 4.97, 0.87)
# examples & price mean & std for validation set within cluster 0 is:(672, 4.99, 0.88)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


Accuracy cluster 0: 0.862336912 (0.010722645)
--------Finished analyzing cluster 0--------
[1 5 1 ... 2 5 7]
--------analyzing cluster 1--------
# examples & price mean & std for training set within cluster 1 is:(6170, 3.78, 1.17)
# examples & price mean & std for validation set within cluster 1 is:(1280, 3.82, 0.50)


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: `np.float` is a deprecated alias for the built

Accuracy cluster 1: 0.815075246 (0.043648696)
--------Finished analyzing cluster 1--------
[1 5 1 ... 2 5 7]
--------analyzing cluster 2--------
# examples & price mean & std for training set within cluster 2 is:(7256, 4.11, 1.24)
# examples & price mean & std for validation set within cluster 2 is:(1525, 4.08, 1.70)


...

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: `np.float` is a deprecated alias for the built

Accuracy cluster 6: 0.860968533 (0.006567236)
--------Finished analyzing cluster 6--------
[1 5 1 ... 2 5 7]
--------analyzing cluster 7--------
# examples & price mean & std for training set within cluster 7 is:(5533, 4.85, 1.16)
# examples & price mean & std for validation set within cluster 7 is:(1150, 4.83, 0.83)


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: `np.float` is a deprecated alias for the built

Accuracy cluster 7: 0.752739765 (0.211724639)
--------Finished analyzing cluster 7--------
--------------------TEST--------------------
Mean absolute error:  0.20770829837204224
Median absolute error:  0.15142961851258363
Mean squared error:  0.18659114723152595
R2:  0.8722345529417777
--------------------TRAIN--------------------
Mean absolute error:  0.20640542214006977
Median absolute error:  0.15142961851258363
Mean squared error:  0.17187795851916385
R2:  0.8721848388798521


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  "X does not have valid feature names, but"
